In [ ]:
from analysis_utils import main
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
from package.orm_models import local_create_session, Result
from itertools import groupby
import sys
from collections import Counter
import pandas as pd
from package.orm_models import Result
import numpy as np
import re

In [ ]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (14, 9)

In [ ]:
def std_show_results(args):
    pass
    sns.heatmap(main(args+['--confusion-matrix']), annot=True, fmt='.0f')
    plt.show()
    main(args+['--plot-correlation'])

In [ ]:
DATABASE = 'grouping.db'
default_args = [
    '--db-file', DATABASE,
]

In [ ]:
session = local_create_session(DATABASE)

In [ ]:
results = session.query(Result).all()
group_results = groupby(results, key=lambda x: list(map(str, x.args)))

results_df = []
results_df_indexes = []
results_df_columns = []
# selection_writter = pd.ExcelWriter('selected_columns.xlsx')
with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
    for key, group in group_results:
        results = list(group)
        result = results[0]
        adj_rand = float(result.adjusted_rand_score)
        experiment_name = 'UNKNOWN'
        scenario = ''
        for arg in key:
            print(f'{arg}')
            if str(arg).startswith('experiment_name'):
                experiment_name = str(arg)[17:]
            elif str(arg).startswith('scenario'):
                scenario = str(arg)[10:]
        selected_features = []
        accuracies = []
        aris = []
        f_measures = []
        for result in results:
            accuracies += [result.accuracy]
            aris += [result.adjusted_rand_score]
            f_measures += [result.f_measure]
            print(f'\tAccuracy: {result.accuracy}')
            print(f'\tARI: {result.adjusted_rand_score}')
            print(f'\tF-Measure: {result.f_measure}')
            selected_features += result.selected_features
        print(f'Accuracy: {np.average(accuracies)} ± {np.std(accuracies)}')
        print(f'ARI: {np.average(aris)} ± {np.std(aris)}')
        print(f'F-Measure: {np.average(f_measures)} ± {np.std(f_measures)}')
        print(f'Selected features: {len(result.selected_features)}')
        selected_features = [feature.column for feature in selected_features]
        selected_features = dict(Counter(selected_features))
        selected_features = pd.DataFrame.from_dict(selected_features, orient='index', columns=['count'])
        selected_features = selected_features.sort_values(by='count', ascending=False)
#         display(selected_features)
#         selected_features.to_excel(selection_writter, experiment_name)
        results_df += [[np.average(accuracies), np.std(accuracies),
                       np.average(aris), np.std(aris),
                       np.average(f_measures), np.std(f_measures)]]
        results_df_indexes += [(experiment_name, scenario)]

        args = default_args + ['--id'] + [str(result.id) for result in results]
        std_show_results(args)
        print('-------------------------------------------------------------------------------------')
        sys.stdout.flush()
        
# selection_writter.save()

In [ ]:
# print(results_df_indexes)
# print(results_df_columns)
# print(results_df)
from itertools import product
index = pd.MultiIndex.from_tuples(results_df_indexes, names=['dataset', 'scenario'])
columns = pd.MultiIndex.from_tuples(product(['Accuracy', 'ARI', 'F-Measure'], ['average', 'std']))
print(columns)
df = pd.DataFrame(results_df, index=results_df_indexes, columns=columns).sort_index()
df.index = pd.MultiIndex.from_tuples(df.index)

In [ ]:
df.to_excel('resultados.xlsx')

In [ ]:
session.close()